In [ ]:
!pip install minsearch

In [ ]:
import minsearch

In [ ]:
import json

In [ ]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [ ]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [ ]:
documents[0]

In [ ]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [ ]:
q = 'the course has already started, can I still enroll?'

In [ ]:
pip install google-generativeai

In [ ]:
index.fit(documents)

In [78]:
import google.generativeai as genai
import os

In [79]:
genai.configure(api_key="") #--- Input your Gemini API Key

In [80]:
model = genai.GenerativeModel('gemini-2.0-flash-lite')

In [81]:
response = model.generate_content([
     {"role": "user", "parts": [q]}
])

In [82]:
print(response.text)

Whether or not you can still enroll in a course after it has started depends on a few factors:

*   **The School/Institution's Policy:** This is the most important factor. Most schools have a specific drop/add period, after which enrolling is no longer permitted. However, some schools or individual departments might have policies that allow late enrollment under specific circumstances.
*   **Course Availability:** Is there still space in the course? If the class is full, you likely won't be able to enroll.
*   **Instructor's Permission:** Some instructors have discretion. If the course is not full and the instructor is willing, they might be able to add you.
*   **Type of Course:** Some courses, like online courses, may be more flexible with late enrollment than in-person courses that have already begun.

**Here's what you should do:**

1.  **Check the Official Enrollment Deadlines:** Look at your school's academic calendar or course registration website. Find the drop/add deadline.
2.

In [119]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [120]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [121]:
def llm(prompt):
    response = model.generate_content(
        {"role": "user", "parts": [prompt]}
    )
    return response.text

In [122]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [123]:
rag(query)

'To run Java Kafka, in the project directory, run: java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java.\n\nTo run Python Kafka, create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once): python -m venv env, source env/bin/activate, pip install -r ../requirements.txt.\nTo activate it: source env/bin/activate.\nTo deactivate it: deactivate.\nFor Windows the path is env/Scripts/activate.\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.\n'

In [89]:
rag('the course has already started, can I still enroll?')

"Yes, you can still join the course after the start date. You are still eligible to submit the homeworks even if you don't register.\n"

In [91]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [113]:
from elasticsearch import Elasticsearch

In [114]:
es_client = Elasticsearch('http://localhost:9200')

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [117]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [118]:
from tqdm.auto import tqdm

In [99]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:44<00:00, 21.33it/s]


In [100]:
query = 'I just disovered the course. Can I still join it?'

In [101]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [102]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [103]:
rag(query)

"Yes, you can still join the course even if you don't register. You are eligible to submit the homeworks.\n"

In [153]:
query = "How do I execute a command in a running docker container?"

In [142]:
rag(query)

'You can execute a command in a running Docker container by using the following command: `docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1`.\n'

In [146]:
search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

In [160]:
search_results = es_client.search(index=index_name, body = search_query)

In [173]:
def search_elastic(question):

    # create your search query
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": question,
                        #"fields": ["question^3", "text", "section^0.5"],
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    # pass the search query to the elasticsearch client
    response = es_client.search(index=index_name, body=search_query)

    # iterate through the response and pull out the list with the results
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [174]:
question = "How do I execute a command in a running docker container?"

In [177]:
def rag_elastic(question):
    #results = search(question, 5)
    results = search_elastic(question)
    prompt = build_prompt(question, results)
    answer = llm(prompt)

    return a

In [178]:
results1 = search_elastic(question)
print(results1[2])

{'text': 'You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I copy files from a different folder into docker container’s working directory?', 'course': 'machine-learning-zoomcamp'}


In [ ]:
prompt1 = build_prompt(question, results1)
len(prompt1)

In [180]:
answer1 = llm(prompt1)
print(answer1)

If the container is already running, you can execute a command in the specific container using:
docker ps (find the container-id)
docker exec -it <container-id> bash



In [ ]:
len(answer1)

In [182]:
!pip install tiktoken

  Using cached tiktoken-0.9.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
Using cached tiktoken-0.9.0-cp312-cp312-win_amd64.whl (894 kB)
Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl (273 kB)

   ---------------------------------------- 0/2 [regex]
   ---------------------------------------- 0/2 [regex]
   -------------------- ------------------- 1/2 [tiktoken]
   ---------------------------------------- 2/2 [tiktoken]



In [ ]:
# Get token count using the model's count_tokens method
response = model.count_tokens(prompt1)

# The response object will have a 'total_tokens' attribute
number_of_tokens = response.total_tokens
print(f"Number of tokens in the prompt: {number_of_tokens}")